# Use a pretrained ImageNet model via `scivision`

(Work in Progress)

In this notebook, we will:

1. load a model from the following test repo:
https://github.com/quantumjot/scivision-test-plugin
2. find matching datasets
3. perform a simple model inference

We first import some things from scivision: `default_catalog` is a scivision **catalog** that will let us discover models and datasets, and `load_pretrained_model` provides a convenient way to load and run a model.

In [ ]:
from scivision import default_catalog, load_pretrained_model

## Query the scivision catalog

A scivision catalog is a collection of **models** and **datasources**.

For this example, we have a particular model in mind (https://github.com/quantumjot/scivision-test-plugin), and wish to find datasets compatible with it.  Let's first list all of the models in the *default catalog* (the built-in catalog, distributed as part of scivision).  Our model is included, named 'scivision-test-plugin'.

In [ ]:
default_catalog.models

Next, we identify datasources in the catalog that would be compatible with the model (based on `tasks`, `format` and `labels`/`labels_required`).  Let's use `data-003`, an image of a baby Koala.

In [ ]:
compatible_datasources = default_catalog.compatible_datasources('scivision-test-plugin')
compatible_datasources

(To be replaced by a query-based interface on the catalog)

In [ ]:
target_datasource = compatible_datasources.loc[compatible_datasources['name'] == 'data-003']
target_datasource

## Load a model

Each model contains a `scivision` config file, for example, see: https://github.com/quantumjot/scivision-test-plugin/blob/main/.scivision-config_imagenet.yaml

There are different ways to specify the path to the config file:

```python
config_path = target['model'] + "/" + '.scivision-config_imagenet.yaml'
```

`scivision` will construct the correct github path.

Or:

```python
config_path = "https://github.com/quantumjot/scivision-test-plugin/blob/main/.scivision-config_imagenet.yaml"
```

Or:

```python
config_path = "https://raw.githubusercontent.com/quantumjot/scivision-test-plugin/main/.scivision-config_imagenet.yaml"
```

In [ ]:
config_path = "https://github.com/quantumjot/scivision-test-plugin/blob/main/.scivision-config_imagenet.yaml"

In [ ]:
model = load_pretrained_model(config_path, allow_install=True)

In [ ]:
# let's explore the model object
model

## Load input image

In [ ]:
# libraries
from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras.applications.imagenet_utils import decode_predictions

import matplotlib.pyplot as plt

In [ ]:
inputs = target_datasource['url'].item()
inputs

In [ ]:
x = imread(inputs)
plt.imshow(x)

In [ ]:
# preprocess: resize to (224, 224) and cast back to 0-255 range
X = resize(x, (224, 224), 
           preserve_range=True, 
           anti_aliasing=True)

## Model predictions

In [ ]:
y = model.predict(X)

In [ ]:
def get_imagenet_label(probs):
    return decode_predictions(probs, top=1)[0][0]

In [ ]:
plt.figure()
plt.imshow(x)
_, image_class, class_confidence = get_imagenet_label(y)
plt.title("{} : {:.2f}%".format(image_class, class_confidence * 100))
plt.show()